In [1]:
# Import packages
import numpy as np
import cvxpy as cp
import mosek
import time
import math
from sklearn.model_selection import train_test_split

import phi_divergence as phi
import robust_sampling as rs
import dataio
import util
import scipy.stats
from scipy.special import erfinv

C:\Users\JSTARRE\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


The problem we examine is as follows (from Esfahani, P., & Kuhn, D. (2018)):


\begin{align}
\label{math_form:examples:pm_exp}
    \min_{\mathbf{x}}&~\mathbb{E}( - \mathbf{r}^T \mathbf{x} ) + \rho \text{CVaR}_{\alpha}(- \mathbf{r}^T \mathbf{x}) \\
    \text{s.t.}&~\mathbf{e}^T \mathbf{x} = 1, \\
    &~\mathbf{x} \geq 0,
\end{align}


where $\mathbf{x}, \mathbf{r} \in \mathbb{R}^{k}$

We follow their derivation and define $J$ as:

$$J = \max_{k=\{1,2\}} a_k \mathbf{r}^T \mathbf{x} + b_k \tau$$,

with $a_1=-1, a_2=-1-\frac{\rho}{\alpha}, b_1=\rho$ and $b_2=\rho\left(1-\frac{1}{\alpha}\right)$.

Then the objective is: $\min_{\mathbf{x}}~\mathbb{E}[J]$. Which can be transformed to epigraph formulation:
\begin{align}
\label{}
    \min_{\mathbf{x}, \tau}&~\theta \\
    \text{s.t.}&~\mathbb{E}[J] \leq \theta,
\end{align}


Filling in the definition of $J$ using variable $$\gamma = J = \max_{k=\{1,2\}} a_k \mathbf{r}^T \mathbf{x} + b_k \tau$$ 

to replace the $\max$ function, we rewrite the problem as follows:

\begin{align}
\label{math_form:examples:pm_exp_2}
    \min_{\mathbf{x}, \tau}&~\theta \\
    \text{s.t.}&~\mathbb{E}[\gamma] \leq \theta, \\
    &~- \mathbf{r}^T \mathbf{x} + \rho \tau \leq \gamma, \\
    &~(-1-(\rho / \alpha)) \mathbf{r}^T \mathbf{x} + \rho(1- (1 / \alpha)) \tau \leq \gamma, \\
    &~\mathbf{e}^T \mathbf{x} = 1, \\
    &~\mathbf{x} \geq 0, \\
    &~\tau, \theta, \gamma \in \mathbb{R},
\end{align}

Thus we have an expected value constraint of the form $\mathbb{E}[f(\mathbf{r}, \mathbf{x}^*)] \leq 0$, with $f(\mathbf{r}, \mathbf{x}^*) = \gamma - \theta = \max_{k=\{1,2\}} \{a_k \mathbf{r}^T \mathbf{x} + b_k \tau \} - \theta$

In their numerical experiments, they set $k = 10$ and generate return by assuming normally distributed systematic + unsystematic risk factors. Furthermore, they set $\alpha = 0.2$ and $\rho = 10$. 


In [2]:
# Problem specific functions:
def generate_data(random_seed, m, N):
    '''
    Taken from:
    https://nbviewer.org/github/MOSEK/Tutorials/blob/master/dist-robust-portfolio/Data-driven_distributionally_robust_portfolio.ipynb
    '''
    np.random.seed(random_seed)
    R = np.vstack([np.random.normal(
        i*0.03, np.sqrt((0.02**2+(i*0.025)**2)), N) for i in range(1, m+1)])
    return (R.transpose())

def generate_data_2(random_seed, m, N):
    np.random.seed(random_seed)
    # NOTE: not entirely clear in Esfahani & Kuhn paper whether they refer to stdev or var
    sys_risk_mean = 0
    #sys_risk_stdev = math.sqrt(0.02)
    sys_risk_stdev = 0.02
    unsys_risk_mean = np.fromiter(((i * 0.03) for i in range(1,k+1)), float)
    #unsys_risk_stdev = np.fromiter(( math.sqrt(i * 0.025) for i in range(1,k+1)), float)
    unsys_risk_stdev = np.fromiter(( i * 0.025 for i in range(1,k+1)), float)
    
    data = np.empty([N,k])
    for n in range(0, N):
        sys_return = np.random.normal(sys_risk_mean, sys_risk_stdev)
        for i in range(0, k):
            unsys_return = np.random.normal(unsys_risk_mean[i], unsys_risk_stdev[i])
            data[n, i] = sys_return + unsys_return
            
    return data 

def solve_SCP(k, S, time_limit):
    x = cp.Variable(k, nonneg = True)
    theta = cp.Variable(1)
    gamma = cp.Variable(1)
    rho = 10 #TODO: fix this hardcode
    CVaR_alpha = 0.20
    tau = cp.Variable(1)
    a_1 = -1
    b_1 = rho
    a_2 = -1 - (rho/CVaR_alpha)
    b_2 = rho*(1 - (1/CVaR_alpha))
    
    constraints = [gamma - theta <= 0,
                   a_1*(S @ x) + b_1*tau - gamma <= 0, 
                   a_2*(S @ x) + b_2*tau - gamma <= 0, 
                   cp.sum(x) == 1]
    
    obj = cp.Maximize(-theta) #equivalent to min \theta
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK, mosek_params = {mosek.dparam.optimizer_max_time: time_limit})
    x_value = np.concatenate((theta.value,gamma.value,tau.value,x.value)) # Combine x, tau and theta into 1 single solution vector
    return(x_value, prob.value)

def uncertain_constraint(S, x):
    rho = 10 #TODO: fix this hardcode
    CVaR_alpha = 0.20
    # Assume that x[1] contains gamma and x[2] contains tau
    # Contrary to other applications, we have 2 uncertain constraints (because of max), the max{} should be <= 0
    constr1 = -np.dot(S,x[3:]) + rho*x[2] - x[1]
    constr2 = -(1+(rho/CVaR_alpha))*np.dot(S,x[3:]) + (rho*(1-(1/CVaR_alpha))*x[2]) - x[1] 
    return np.maximum(constr1, constr2)

def check_robust(bound, numeric_precision, beta=0):
    return (bound <= beta + numeric_precision)

def emp_eval_obj(x, data, rho, CVaR_alpha):
    x_sol = x[3:]
    emp_returns = - (np.dot(data, x_sol))

    exp_loss = np.mean(emp_returns, dtype=np.float64)
    # print(exp_loss)
    
    VaR = np.quantile(emp_returns, 1-CVaR_alpha, method='inverted_cdf') # gets threshold for top CVaR_alpha-% highest losses
    above_VaR = (emp_returns > VaR)
    cVaR = np.mean(emp_returns[above_VaR])
    # print(cVaR)
    
    return exp_loss + (rho*cVaR)

def analytic_out_perf(x, rho, CVaR_alpha):
    '''
    https://nbviewer.org/github/MOSEK/Tutorials/blob/master/dist-robust-portfolio/Data-driven_distributionally_robust_portfolio.ipynb
    Method to calculate the analytical value for the out-of-sample performance.
    [see Rockafellar and Uryasev]
    '''
    x_sol = x[3:]
    m = len(x_sol)
    mu = np.arange(1, m+1)*0.03
    var = 0.02 + (np.arange(1, m+1)*0.025)

    # Constants for CVaR calculation.
    rho = 10
    beta = 1-CVaR_alpha
    c2_beta = 1/(np.sqrt(2*np.pi)*(np.exp(erfinv(2*beta - 1))**2)*(1-beta))
    
    mean_loss = -np.dot(x_sol, mu)
    # print(mean_loss)
    sd_loss = np.sqrt(np.dot(x_sol**2, var))
    cVaR = mean_loss + (sd_loss*c2_beta)
    # print(cVaR)
    return mean_loss + (rho*cVaR)

In [3]:
def solve_SCP_SAA(k, S, time_limit):
    # hardcoded for now...
    rho = 10 
    CVaR_alpha = 0.20
    a_1 = -1
    b_1 = rho
    a_2 = -1 - (rho/CVaR_alpha)
    b_2 = rho*(1 - (1/CVaR_alpha))
    N = S.shape[0]
    
    x = cp.Variable(k, nonneg = True)
    theta = cp.Variable(1)
    gamma = cp.Variable(N)
    tau = cp.Variable(1)
    
    constraints = [(1/N)*cp.sum(gamma) - theta <= 0,
                   a_1*(S @ x) + b_1*tau - gamma <= 0, 
                   a_2*(S @ x) + b_2*tau - gamma <= 0, 
                   cp.sum(x) == 1]
    
    obj = cp.Maximize(-theta) #equivalent to min \theta
    prob = cp.Problem(obj,constraints)
    try:
        prob.solve(solver=cp.MOSEK, 
                   # verbose=True,
                   mosek_params = {mosek.dparam.optimizer_max_time: time_limit}
                   )
    except cp.error.SolverError:
        print("Note: error occured in solving SAA problem...")
        return(None, float('-inf'))
    # Combine x, tau and theta into 1 single solution vector
    x_value = np.concatenate((theta.value,np.array([None]),tau.value,x.value))
    return(x_value, prob.value)

def uncertain_constraint_SAA(S, x):
    # hardcoded for now...
    rho = 10
    CVaR_alpha = 0.20
    a_1 = -1
    b_1 = rho
    a_2 = -1 - (rho/CVaR_alpha)
    b_2 = rho*(1 - (1/CVaR_alpha))
    N = S.shape[0]
    
    # Assume that x[1] contains gamma (vec of length N) and x[2] contains tau
    # Contrary to other applications, we have 2 uncertain constraints (because of max)
    # the max should be <= 0
    constr1 = a_1*(S @ x[3:]) + b_1*x[2]
    constr2 = a_2*(S @ x[3:]) + b_2*x[2]
    return np.maximum(constr1, constr2) - x[0]

In [5]:
# SCP vs SAA
k = 10
rho = 10
CVaR_alpha = 0.20

for N in [10, 100, 1000, 10000, 100000]:
    data = generate_data_2(0, k, N)
    sol, obj = solve_SCP(k, data, 10*60)
    print(round(-obj,3), round(analytic_out_perf(sol, rho, CVaR_alpha),3))
    
print('-----------------------------')
for N in [10, 100, 1000, 10000, 100000]:
    data = generate_data_2(0, k, N)
    sol, obj = solve_SCP_SAA(k, data, 10*60)
    print(round(-obj,3), round(analytic_out_perf(sol, rho, CVaR_alpha),3))
    

-1.833 -0.864
-0.743 -1.139
-0.27 -0.866
0.178 -0.425
0.394 -0.432
-----------------------------
-1.951 -0.864
-1.344 -1.353
-1.312 -1.356
-1.364 -1.379
-1.363 -1.387


In [ ]:
# test obj eval functions with equal weighted portfolio:
k = 10
x = np.array([None, None, None] + [1/k for i in range(k)])
rho = 10
CVaR_alpha = 0.20

for N in [1000000]:
    data = generate_data(66 + 99, k, N)
    obj_1 = emp_eval_obj(x, data, rho, CVaR_alpha)
    print(obj_1)
    
    print("----------------")
    data_2 = generate_data_2(66 + 99, k, N)
    obj_2 = emp_eval_obj(x, data_2, rho, CVaR_alpha)
    print(obj_2)
    
    print("----------------")
    obj_3 = analytic_out_perf(x, rho, CVaR_alpha)
    print(obj_3)

# data gen 2 seems more accurate (if analytic_out_perf is correct)
# Not sure why there is still a slight difference... 
# suspect that there is a mistake in analytic cVaR because empirical evaluation seems sound

In [ ]:
# Set parameter values (as in Kuhn paper)
k = 10
rho = 10
CVaR_alpha = 0.20
risk_measure = 'exp_constraint_leq' # options: 'chance_constraint', 'exp_constraint'
alpha = 0.50
beta = 0
N_total = 10000 # 30, 300, 3000
N_train = int(N_total / 2)
N_test = N_total - N_train
num_obs_per_bin = max(N_test / 10, 5)

In [ ]:
# Set other parameter values
par = 1
phi_div = phi.mod_chi2_cut
phi_dot = 2
numeric_precision = 1e-6 # To correct for floating-point math operations

In [ ]:
# Get generated data
random_seed = 0
data = generate_data(random_seed, k, N_total)   
data_train, data_test = train_test_split(data, train_size=(N_train/N_total), random_state=random_seed)

In [ ]:
#OPTIONAL:
N_eval = 100000
data_eval = generate_data(random_seed + 99, k, N_eval)

In [ ]:
time_limit_search = 1*60 # in seconds (time provided to search algorithm)
time_limit_mosek = 10*60 # in seconds (for larger MIP / LP solves)
time_limit_solve = 10*60 # in seconds (for individuals solves of SCP)
max_nr_solutions = 1 # for easy problems with long time limits, we may want extra restriction
add_remove_threshold = 0.00 # This determines when randomness is introduced in add/removal decision
use_tabu = False # Determines whether the tabu list are used in the search

add_strategy = 'random_vio'
remove_strategy = 'random_any'
clean_strategy = (999999, 'all_inactive') # set arbitrarily high such that it never occurs

# Alters the Sampled Convex Problem
solve_SCP = solve_SCP_SAA
uncertain_constraint = uncertain_constraint_SAA

(runtime, num_iter, solutions, 
 best_sol, pareto_solutions) = rs.gen_and_eval_alg(data_train, data_test, beta, alpha, time_limit_search, time_limit_solve, 
                                                    max_nr_solutions, add_strategy, remove_strategy, clean_strategy, 
                                                    add_remove_threshold, use_tabu,
                                                    phi_div, phi_dot, numeric_precision,
                                                    solve_SCP, uncertain_constraint, check_robust,
                                                    risk_measure, random_seed, 
                                                   num_obs_per_bin, data_eval, emp_eval_obj,
                                                  analytic_out_perf)

In [ ]:
x = best_sol['sol']
x

In [ ]:
# Solve SCP with many data points in set
sol, obj = solve_SCP(k, data_eval, 10*60)
x = sol
x

In [ ]:
sol

In [ ]:
# Portfolio with equal weights
x = np.array([None, None, None] + [1/k for i in range(k)])

In [ ]:
# Evaluate obj emperically using data_eval
emp_eval_obj(x, data_eval, rho, 0.20)

In [ ]:
# Evaluate obj analytically
analytic_out_perf(x, rho, CVaR_alpha)

In [ ]:
num_iter

In [ ]:
x_plot = x[3:]
dataio.plot_single_portfolio_holdings(x_plot)

In [ ]:
dataio.plot_pareto_curve(pareto_solutions, beta, None, None, None, None)